## 1. 問題の分析

### 競技プログラミング視点での分析

* **狙い**: すべてのノードが「(low, high) の範囲内にある」ことを **1回の走査で検証**すれば十分。
* **最適方針**: 各ノードに許容範囲（下限・上限）を持たせて DFS。

  * 速度: 1ノード1回チェック → **O(n)**
  * メモリ: 再帰はスタックオーバーフロー懸念（最大 1e4）→ **反復（iterative）で回避**が堅い。

### 業務開発視点での分析

* **可読性/保守性**: 「BST条件＝範囲制約」として表現すると仕様が明確。
* **エラーハンドリング**: LeetCode入力は型保証される前提だが、業務では `unknown` からの型ガードが必要になりがち。今回は実装内に軽量な型ガードを入れてもよい（ただし通常は不要）。

### TypeScript特有の考慮点

* `strict mode` 前提で `null` を明示し、`TreeNode | null` を徹底。
* 範囲は `number`（`-Infinity / Infinity`）で表すと **境界条件が簡潔**。
* 反復 DFS のスタック要素を `readonly` にして意図しない破壊を防止（実質的な immutability）。

---

## 2. アルゴリズムアプローチ比較

| アプローチ                 | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考                                 |
| --------------------- | ----: | ----: | ------- | ---- | --- | ---------------------------------- |
| 方法A: 再帰DFS（上下限）       |  O(n) |  O(h) | 低       | 高    | 高   | `h` が 1e4 だと Node.js でコールスタック限界リスク |
| 方法B: 反復DFS（上下限）       |  O(n) |  O(h) | 中       | 高    | 高   | **速い/安全（スタック溢れ回避）**、本回答で採用         |
| 方法C: inorder 反復（前値比較） |  O(n) |  O(h) | 中       | 高    | 中   | 単純だが「BST＝inorder昇順」の理解が必要          |
| 方法D: inorder 配列化 → 検証 |  O(n) |  O(n) | 低       | 高    | 高   | メモリ無駄（配列確保）                        |
| 方法E: Morris traversal |  O(n) |  O(1) | 高       | 中    | 低   | 木を一時改変（復元必要）で純粋性/保守性が落ちる           |

---

## 3. 選択したアルゴリズムと理由

* **選択したアプローチ**: 方法B（反復DFS + 上下限）
* **理由**:

  * 計算量が最適（**O(n)**）で、追加メモリも **O(h)** に抑えられる
  * 再帰を避けることで **Node.js のコールスタック制限**に強い
  * 「許容範囲」という形で仕様が表現でき、**可読性と保守性が高い**
* **TypeScript特有の最適化ポイント**:

  * `TreeNode | null` を明示して null 安全
  * スタック要素を `readonly` 構造で表現し、意図しない変更を防止

---

## 4. 実装コード（LeetCode フォーマット / ESM想定）

```typescript
/**
 * Definition for a binary tree node.
 * LeetCode provides this in runtime; included here for completeness.
 */
class TreeNode {
  val: number;
  left: TreeNode | null;
  right: TreeNode | null;
  constructor(val?: number, left?: TreeNode | null, right?: TreeNode | null) {
    this.val = val ?? 0;
    this.left = left ?? null;
    this.right = right ?? null;
  }
}

type StackEntry = Readonly<{
  node: TreeNode;
  low: number;  // node.val must be > low
  high: number; // node.val must be < high
}>;

/**
 * Validate whether a binary tree is a valid BST.
 *
 * - Pure function: does not mutate the tree.
 * - Iterative DFS with (low, high) constraints to avoid recursion depth issues.
 *
 * @param root - Root of the binary tree
 * @returns true if valid BST, otherwise false
 * @throws {TypeError} If input is not a TreeNode-like structure (defensive; usually unnecessary on LeetCode)
 * @complexity Time: O(n), Space: O(h)  (h = tree height; worst-case O(n))
 */
function isValidBST(root: TreeNode | null): boolean {
  // Defensive runtime guard (LeetCode input is trusted; safe in normal runs)
  if (root !== null && !isTreeNode(root)) {
    throw new TypeError("Input must be a TreeNode or null");
  }

  if (root === null) return true;

  const stack: StackEntry[] = [{ node: root, low: -Infinity, high: Infinity }];

  while (stack.length > 0) {
    const cur = stack.pop()!;
    const v = cur.node.val;

    // Strict inequalities are required
    if (!(v > cur.low && v < cur.high)) return false;

    const left = cur.node.left;
    if (left !== null) {
      if (!isTreeNode(left)) throw new TypeError("Invalid left child node");
      stack.push({ node: left, low: cur.low, high: v });
    }

    const right = cur.node.right;
    if (right !== null) {
      if (!isTreeNode(right)) throw new TypeError("Invalid right child node");
      stack.push({ node: right, low: v, high: cur.high });
    }
  }

  return true;
}

/** Type guard for defensive checks */
function isTreeNode(x: unknown): x is TreeNode {
  if (typeof x !== "object" || x === null) return false;
  const obj = x as { val?: unknown; left?: unknown; right?: unknown };
  const valOk = typeof obj.val === "number";
  const leftOk = obj.left === null || typeof obj.left === "object";
  const rightOk = obj.right === null || typeof obj.right === "object";
  return valOk && leftOk && rightOk;
}

Analyze Complexity
Runtime 5 ms
Beats 7.82%
Memory 61.19 MB
Beats 12.52%

```

---

必要なら、同じ計算量で「inorder 反復（前値比較）」版も併記して、レビュー観点（読みやすさ vs 仕様の明示性）で比較できる形にできます。

改善余地はあります。ただし **Beats% は提出タイミングの母集団・JITの揺れでブレる**ので、`5ms` 自体は十分速い部類です。メモリも TS 実行環境のオーバーヘッドが大きく、アルゴリズム差が Beats に反映されにくいことが多いです。

それでも「改善しやすい」ポイントは2つあります。

## 改善ポイント

1. **(low, high) を持つ方式はスタック要素が重い**
   `[node, low, high]` を毎回 push するとタプル生成＋保持が増え、メモリが不利になりがち。

2. **inorder 反復（前値比較）にすると保持データがノードだけ**
   BST の inorder は「厳密増加」になるので、`prev` だけ見れば良いです。
   → スタックは `TreeNode[]` のみになり、メモリが減りやすいです。

---

## より軽量な実装（おすすめ：inorder 反復）

```typescript
function isValidBST(root: TreeNode | null): boolean {
  const stack: TreeNode[] = [];
  let cur: TreeNode | null = root;
  let prev: number | null = null;

  while (cur !== null || stack.length > 0) {
    while (cur !== null) {
      stack.push(cur);
      cur = cur.left;
    }

    const node = stack.pop()!;
    const v = node.val;

    if (prev !== null && v <= prev) return false;
    prev = v;

    cur = node.right;
  }

  return true;
}

Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 60.80 MB
Beats 15.12%

```

### これが効く理由

* push/pop するのが **ノード参照だけ**（`low/high` の数値を抱えない）
* **タプルの生成・分割代入**もないので、GC負荷が減りやすい

---

## さらに微改善したい場合（小技）

* 分割代入（`const [a,b,c] = ...`）やタプル生成を避ける（今回の inorder は既に回避済み）
* 不要なガード/例外処理を入れない（LeetCode前提なら最小コードが速いことが多い）

まずは上の inorder 版に差し替えるのが、**メモリ改善に一番効きやすい**です。